In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib inline
import sys
import os
import numpy as np
import pandas as pd
import scipy.sparse
from scipy.sparse import csc_matrix
from scipy.sparse import lil_matrix

import matplotlib.pyplot as plt
import matplotlib.animation as animation
from mpl_toolkits.basemap import Basemap
from matplotlib.patches import Circle
import seaborn as sns; 
from IPython.display import HTML

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from sklearn import svm
from sklearn.metrics import r2_score
from statistics import mean

In [ ]:
#config parser
import configparser

sys.path.insert(0, '../Common/')
from AISDataManager import AISDataManager
import Constants as c
import HMUtils as hMUtil
import TimeUtils as timeUtils
import GeoCompute as gC

#MyConfig.INI stores all the run time constants
config = configparser.ConfigParser()
config.read('../MyConfig.INI')

from joblib import Parallel, delayed
import multiprocessing
aISDM = AISDataManager()
numCores = multiprocessing.cpu_count()

In [ ]:
lonMin = (float)(config['TP_SEC_ORDER']['LON_MIN'])
lonMax = (float)(config['TP_SEC_ORDER']['LON_MAX'])

latMin = (float)(config['TP_SEC_ORDER']['LAT_MIN'])
latMax = (float)(config['TP_SEC_ORDER']['LAT_MAX'])

print(lonMin,latMin)
print(lonMax,latMax)

increStep = (float)(config['TP_SEC_ORDER']['INCR_STEP'])
incrRes = (int)(config['TP_SEC_ORDER']['INCR_RES'])

fileDir = config['TP_SEC_ORDER']['SOURCE_DIR']
dirToStore = config['TP_SEC_ORDER']['DEST_DIR']

randStart = (int)(config['ERROR_PLOT']['RAND_START'])
randEnd = (int)(config['ERROR_PLOT']['RAND_END'])
trajDir = (config['ERROR_PLOT']['DATA_DIR'])
errorHistDest = (config['ERROR_PLOT']['ERROR_HIST_DEST'])
erroPlotName = (config['ERROR_PLOT']['ERROR_PLOT_DEST'])

print("SOURCE_DIR = %s"%fileDir)
print("DEST_DIR = %s"%dirToStore)
print("TRAJ_DIR = %s"%trajDir)
print("ERROR_HIST_DIR = %s"%errorHistDest)
print("ERROR_PLOT_NAME = %s"%erroPlotName)

In [ ]:
heatMapGrid = hMUtil.generate_grid(lonMin, lonMax, latMin, latMax, increStep, incrRes)
boundaryArray = heatMapGrid[2]
horizontalAxis = heatMapGrid[0]
verticalAxis = heatMapGrid[1]
totalStates = horizontalAxis.shape[0] * verticalAxis.shape[0]

opFile = dirToStore + 'TPM.npz'
tpMatrix = scipy.sparse.load_npz(opFile)

In [ ]:
def get_index_from_lon_lat(lon,lat):
    retVal = -1
    for boundary in boundaryArray: 
        if(lon >= boundary[0]) and (lon < boundary[1]) \
            and (lat >= boundary[2]) and (lat < boundary[3]):
            retVal = boundary[4]
            break 
    return retVal

def get_vessel_init_status(lonStart,latStart):
    vesselState = np.zeros((horizontalAxis.shape[0]*verticalAxis.shape[0]))
    for boundary in boundaryArray :
        if((lonStart >= boundary[0]) \
            and (lonStart < boundary[1]) \
           
            and (latStart >= boundary[2]) \
            and (latStart < boundary[3]) \
          ):
            vesselState[boundary[4]] = 1.0
            break
    return vesselState

#takes inital state vector
#and number of steps 
#uses transition probabilities to generate the path
def generate_path(vesselStatus):
    return (vesselStatus.T @ tpMatrix).flatten()

def generate_vessel_prev_cur_status(prevState,currState):
    vesselState = np.zeros((totalStates,totalStates))
    for i in range(prevState.shape[0]):
        vesselState[i,:] = (prevState[i]*currState)
    return vesselState.flatten()

In [ ]:
#get random list of 500 vessels
#for error compute
#set the seed
np.random.seed(0)
usefulList = np.random.randint(randStart,randEnd,size = 1000).tolist()

In [ ]:
print(usefulList)

In [ ]:
rowSum = np.sum(tpMatrix, axis=1)
rowSumZeroIdx = np.argwhere(rowSum == 0)
rowSumZeroIdx = rowSumZeroIdx.flatten()

In [ ]:
errorVal30_1 = []
errorVal30_2 = []
errorVal30_3 = []
errorVal30_4 = []
errorVal30_5 = []
errorVal30_6 = []
errorVal30_7 = []
errorVal30_8 = []
errorVal30_9 = []
errorVal30_10 = []

errorVal60_1 = []
errorVal60_2 = []
errorVal60_3 = []
errorVal60_4 = []
errorVal60_5 = []
errorVal60_6 = []
errorVal60_7 = []
errorVal60_8 = []
errorVal60_9 = []
errorVal60_10 = []

errorVal90_1 = []
errorVal90_2 = []
errorVal90_3 = []
errorVal90_4 = []
errorVal90_5 = []
errorVal90_6 = []
errorVal90_7 = []
errorVal90_8 = []
errorVal90_9 = []
errorVal90_10 = []

errorVal120_1 = []
errorVal120_2 = []
errorVal120_3 = []
errorVal120_4 = []
errorVal120_5 = []
errorVal120_6 = []
errorVal120_7 = []
errorVal120_8 = []
errorVal120_9 = []
errorVal120_10 = []

errorVal150_1 = []
errorVal150_2 = []
errorVal150_3 = []
errorVal150_4 = []
errorVal150_5 = []
errorVal150_6 = []
errorVal150_7 = []
errorVal150_8 = []
errorVal150_9 = []
errorVal150_10 = []

errorVal180_1 = []
errorVal180_2 = []
errorVal180_3 = []
errorVal180_4 = []
errorVal180_5 = []
errorVal180_6 = []
errorVal180_7 = []
errorVal180_8 = []
errorVal180_9 = []
errorVal180_10 = []

errorVal210_1 = []
errorVal210_2 = []
errorVal210_3 = []
errorVal210_4 = []
errorVal210_5 = []
errorVal210_6 = []
errorVal210_7 = []
errorVal210_8 = []
errorVal210_9 = []
errorVal210_10 = []

errorVal240_1 = []
errorVal240_2 = []
errorVal240_3 = []
errorVal240_4 = []
errorVal240_5 = []
errorVal240_6 = []
errorVal240_7 = []
errorVal240_8 = []
errorVal240_9 = []
errorVal240_10 = []

for lEFileNum in usefulList:
    print(lEFileNum)
    oneVessel = trajDir+str(lEFileNum)+".csv"
    oneVesselLastData,_ = aISDM.load_data_from_csv(oneVessel)
    oneVesselLastDataLon = oneVesselLastData["LON"]
    oneVesselLastDataLat = oneVesselLastData["LAT"]
    plt.plot(oneVesselLastDataLon,oneVesselLastDataLat)

    vesselPrevState = get_vessel_init_status(oneVesselLastDataLon.iloc[0],oneVesselLastDataLat.iloc[0])
    vesselCurrState = get_vessel_init_status(oneVesselLastDataLon.iloc[1],oneVesselLastDataLat.iloc[1])
    vesselState = generate_vessel_prev_cur_status(vesselPrevState,vesselCurrState)
    
    onePredict = []
#     for i in range(8):
    for i in range(oneVesselLastDataLon.shape[0]-2):
        vesselProbState = generate_path(vesselState)
        onePredict.append(vesselProbState.copy())
        vesselPrevState = vesselCurrState.copy()
        vesselCurrState = vesselProbState.copy()
        vesselState = generate_vessel_prev_cur_status(vesselPrevState,vesselCurrState)
        vesselState[rowSumZeroIdx] = 0
        print(np.sum(vesselState))
        if(np.sum(vesselState) == 0):
            print("No trajectory instance found")
            break
        else:
            vesselState = vesselState/np.sum(vesselState)


            actualLastLoc = get_index_from_lon_lat(oneVesselLastDataLon.iloc[2+i],oneVesselLastDataLat.iloc[2+i])
            actualLastLocCord = hMUtil.compute_mid_point(boundaryArray[actualLastLoc][0] \
                                                            ,boundaryArray[actualLastLoc][1]\
                                                            ,boundaryArray[actualLastLoc][2]\
                                                            ,boundaryArray[actualLastLoc][3]\
                                                            )
            #this is an array of top 10
            predictedLastLoc = onePredict[-1].argsort()[-10:][::-1]
#             print(predictedLastLoc)
            #top 1
            predictedLastLocCord = hMUtil.compute_mid_point(boundaryArray[predictedLastLoc[0]][0] \
                                                            ,boundaryArray[predictedLastLoc[0]][1]\
                                                            ,boundaryArray[predictedLastLoc[0]][2]\
                                                            ,boundaryArray[predictedLastLoc[0]][3]\
                                                            )

            errorPos1 = gC.compute_distance(predictedLastLocCord[0], predictedLastLocCord[1], actualLastLocCord[0], actualLastLocCord[1])

            #top 2
            predictedLastLocCord = hMUtil.compute_mid_point(boundaryArray[predictedLastLoc[1]][0] \
                                                            ,boundaryArray[predictedLastLoc[1]][1]\
                                                            ,boundaryArray[predictedLastLoc[1]][2]\
                                                            ,boundaryArray[predictedLastLoc[1]][3]\
                                                            )

            errorPos2 = gC.compute_distance(predictedLastLocCord[0], predictedLastLocCord[1], actualLastLocCord[0], actualLastLocCord[1])

            #top 3
            predictedLastLocCord = hMUtil.compute_mid_point(boundaryArray[predictedLastLoc[2]][0] \
                                                            ,boundaryArray[predictedLastLoc[2]][1]\
                                                            ,boundaryArray[predictedLastLoc[2]][2]\
                                                            ,boundaryArray[predictedLastLoc[2]][3]\
                                                            )

            errorPos3 = gC.compute_distance(predictedLastLocCord[0], predictedLastLocCord[1], actualLastLocCord[0], actualLastLocCord[1])

            #top 4
            predictedLastLocCord = hMUtil.compute_mid_point(boundaryArray[predictedLastLoc[3]][0] \
                                                            ,boundaryArray[predictedLastLoc[3]][1]\
                                                            ,boundaryArray[predictedLastLoc[3]][2]\
                                                            ,boundaryArray[predictedLastLoc[3]][3]\
                                                            )

            errorPos4 = gC.compute_distance(predictedLastLocCord[0], predictedLastLocCord[1], actualLastLocCord[0], actualLastLocCord[1])

            #top 5
            predictedLastLocCord = hMUtil.compute_mid_point(boundaryArray[predictedLastLoc[4]][0] \
                                                            ,boundaryArray[predictedLastLoc[4]][1]\
                                                            ,boundaryArray[predictedLastLoc[4]][2]\
                                                            ,boundaryArray[predictedLastLoc[4]][3]\
                                                            )

            errorPos5 = gC.compute_distance(predictedLastLocCord[0], predictedLastLocCord[1], actualLastLocCord[0], actualLastLocCord[1])

            #top 6
            predictedLastLocCord = hMUtil.compute_mid_point(boundaryArray[predictedLastLoc[5]][0] \
                                                            ,boundaryArray[predictedLastLoc[5]][1]\
                                                            ,boundaryArray[predictedLastLoc[5]][2]\
                                                            ,boundaryArray[predictedLastLoc[5]][3]\
                                                            )

            errorPos6 = gC.compute_distance(predictedLastLocCord[0], predictedLastLocCord[1], actualLastLocCord[0], actualLastLocCord[1])

            #top 7
            predictedLastLocCord = hMUtil.compute_mid_point(boundaryArray[predictedLastLoc[6]][0] \
                                                            ,boundaryArray[predictedLastLoc[6]][1]\
                                                            ,boundaryArray[predictedLastLoc[6]][2]\
                                                            ,boundaryArray[predictedLastLoc[6]][3]\
                                                            )

            errorPos7 = gC.compute_distance(predictedLastLocCord[0], predictedLastLocCord[1], actualLastLocCord[0], actualLastLocCord[1])

            #top 8
            predictedLastLocCord = hMUtil.compute_mid_point(boundaryArray[predictedLastLoc[7]][0] \
                                                            ,boundaryArray[predictedLastLoc[7]][1]\
                                                            ,boundaryArray[predictedLastLoc[7]][2]\
                                                            ,boundaryArray[predictedLastLoc[7]][3]\
                                                            )

            errorPos8 = gC.compute_distance(predictedLastLocCord[0], predictedLastLocCord[1], actualLastLocCord[0], actualLastLocCord[1])

            #top 9
            predictedLastLocCord = hMUtil.compute_mid_point(boundaryArray[predictedLastLoc[8]][0] \
                                                            ,boundaryArray[predictedLastLoc[8]][1]\
                                                            ,boundaryArray[predictedLastLoc[8]][2]\
                                                            ,boundaryArray[predictedLastLoc[8]][3]\
                                                            )

            errorPos9 = gC.compute_distance(predictedLastLocCord[0], predictedLastLocCord[1], actualLastLocCord[0], actualLastLocCord[1])

            #top 10
            predictedLastLocCord = hMUtil.compute_mid_point(boundaryArray[predictedLastLoc[9]][0] \
                                                            ,boundaryArray[predictedLastLoc[9]][1]\
                                                            ,boundaryArray[predictedLastLoc[9]][2]\
                                                            ,boundaryArray[predictedLastLoc[9]][3]\
                                                            )

            errorPos10 = gC.compute_distance(predictedLastLocCord[0], predictedLastLocCord[1], actualLastLocCord[0], actualLastLocCord[1])

            if(i == 0):
                errorVal30_1.append(errorPos1)
                errorVal30_2.append(errorPos2)
                errorVal30_3.append(errorPos3)
                errorVal30_4.append(errorPos4)
                errorVal30_5.append(errorPos5)
                errorVal30_6.append(errorPos6)
                errorVal30_7.append(errorPos7)
                errorVal30_8.append(errorPos8)
                errorVal30_9.append(errorPos9)
                errorVal30_10.append(errorPos10)
            elif(i == 1):
                errorVal60_1.append(errorPos1)
                errorVal60_2.append(errorPos2)
                errorVal60_3.append(errorPos3)
                errorVal60_4.append(errorPos4)
                errorVal60_5.append(errorPos5)
                errorVal60_6.append(errorPos6)
                errorVal60_7.append(errorPos7)
                errorVal60_8.append(errorPos8)
                errorVal60_9.append(errorPos9)
                errorVal60_10.append(errorPos10)
            elif(i == 2):
                errorVal90_1.append(errorPos1)
                errorVal90_2.append(errorPos2)
                errorVal90_3.append(errorPos3)
                errorVal90_4.append(errorPos4)
                errorVal90_5.append(errorPos5)
                errorVal90_6.append(errorPos6)
                errorVal90_7.append(errorPos7)
                errorVal90_8.append(errorPos8)
                errorVal90_9.append(errorPos9)
                errorVal90_10.append(errorPos10)
            elif(i == 3):
                errorVal120_1.append(errorPos1)
                errorVal120_2.append(errorPos2)
                errorVal120_3.append(errorPos3)
                errorVal120_4.append(errorPos4)
                errorVal120_5.append(errorPos5)
                errorVal120_6.append(errorPos6)
                errorVal120_7.append(errorPos7)
                errorVal120_8.append(errorPos8)
                errorVal120_9.append(errorPos9)
                errorVal120_10.append(errorPos10)
            elif(i == 4):
                errorVal150_1.append(errorPos1)
                errorVal150_2.append(errorPos2)
                errorVal150_3.append(errorPos3)
                errorVal150_4.append(errorPos4)
                errorVal150_5.append(errorPos5)
                errorVal150_6.append(errorPos6)
                errorVal150_7.append(errorPos7)
                errorVal150_8.append(errorPos8)
                errorVal150_9.append(errorPos9)
                errorVal150_10.append(errorPos10)
            elif(i == 5):
                errorVal180_1.append(errorPos1)
                errorVal180_2.append(errorPos2)
                errorVal180_3.append(errorPos3)
                errorVal180_4.append(errorPos4)
                errorVal180_5.append(errorPos5)
                errorVal180_6.append(errorPos6)
                errorVal180_7.append(errorPos7)
                errorVal180_8.append(errorPos8)
                errorVal180_9.append(errorPos9)
                errorVal180_10.append(errorPos10)
            elif(i == 6):
                errorVal210_1.append(errorPos1)
                errorVal210_2.append(errorPos2)
                errorVal210_3.append(errorPos3)
                errorVal210_4.append(errorPos4)
                errorVal210_5.append(errorPos5)
                errorVal210_6.append(errorPos6)
                errorVal210_7.append(errorPos7)
                errorVal210_8.append(errorPos8)
                errorVal210_9.append(errorPos9)
                errorVal210_10.append(errorPos10)
            elif(i == 7):
                errorVal240_1.append(errorPos1)
                errorVal240_2.append(errorPos2)
                errorVal240_3.append(errorPos3)
                errorVal240_4.append(errorPos4)
                errorVal240_5.append(errorPos5)
                errorVal240_6.append(errorPos6)
                errorVal240_7.append(errorPos7)
                errorVal240_8.append(errorPos8)
                errorVal240_9.append(errorPos9)
                errorVal240_10.append(errorPos10) 

In [ ]:
min30_1 =[]
min30_3 =[]
min30_5 =[]
min30_10 =[]

for i in range(len(errorVal30_1)):
    top1Error = errorVal30_1[i]
    
    top3Error = min(errorVal30_1[i] \
                    ,errorVal30_2[i] \
                    ,errorVal30_3[i] \
                   )
    top5Error = min(errorVal30_1[i] \
                    ,errorVal30_2[i] \
                    ,errorVal30_3[i] \
                    ,errorVal30_4[i] \
                    ,errorVal30_5[i] \
                   )
    top10Error = min(errorVal30_1[i] \
                    ,errorVal30_2[i] \
                    ,errorVal30_3[i] \
                    ,errorVal30_4[i] \
                    ,errorVal30_5[i] \
                    ,errorVal30_6[i] \
                    ,errorVal30_7[i] \
                    ,errorVal30_8[i] \
                    ,errorVal30_9[i] \
                    ,errorVal30_10[i] \
                   )
    min30_1.append(top1Error)
    min30_3.append(top3Error)
    min30_5.append(top5Error)
    min30_10.append(top10Error)

In [ ]:
plt.plot(min30_5)

In [ ]:
print(mean(min30_1))
print(mean(min30_3))
print(mean(min30_5))
print(mean(min30_10))

In [ ]:
plt.hist(min30_3)

In [ ]:
plt.hist(min30_5)

In [ ]:
min60_1 =[]
min60_3 =[]
min60_5 =[]
min60_10 =[]

for i in range(len(errorVal60_1)):
    top1Error = errorVal60_1[i]
    
    top3Error = min(errorVal60_1[i] \
                    ,errorVal60_2[i] \
                    ,errorVal60_3[i] \
                   )
    top5Error = min(errorVal60_1[i] \
                    ,errorVal60_2[i] \
                    ,errorVal60_3[i] \
                    ,errorVal60_4[i] \
                    ,errorVal60_5[i] \
                   )
    top10Error = min(errorVal60_1[i] \
                    ,errorVal60_2[i] \
                    ,errorVal60_3[i] \
                    ,errorVal60_4[i] \
                    ,errorVal60_5[i] \
                    ,errorVal60_6[i] \
                    ,errorVal60_7[i] \
                    ,errorVal60_8[i] \
                    ,errorVal60_9[i] \
                    ,errorVal60_10[i] \
                   )
    min60_1.append(top1Error)
    min60_3.append(top3Error)
    min60_5.append(top5Error)
    min60_10.append(top10Error)

In [ ]:
print(mean(min60_1))
print(mean(min60_3))
print(mean(min60_5))
print(mean(min60_10))

In [ ]:
plt.plot(min60_5)

In [ ]:
print(min60_5)

In [ ]:
plt.title("Histogram of Top 3 Error After 60 Minutes")
plt.xlabel("Error in KM")
plt.hist(min60_3)
figName = errorHistDest + "Top3Err60Min.png"
plt.savefig(figName)

In [ ]:
plt.title("Histogram of Top 5 Error After 60 Minutes")
plt.xlabel("Error in KM")
plt.hist(min60_5)
figName = errorHistDest + "Top5Err60Min.png"
plt.savefig(figName)

In [ ]:
min90_1 =[]
min90_3 =[]
min90_5 =[]
min90_10 =[]

for i in range(len(errorVal90_1)):
    top1Error = errorVal90_1[i]
    
    top3Error = min(errorVal90_1[i] \
                    ,errorVal90_2[i] \
                    ,errorVal90_3[i] \
                   )
    top5Error = min(errorVal90_1[i] \
                    ,errorVal90_2[i] \
                    ,errorVal90_3[i] \
                    ,errorVal90_4[i] \
                    ,errorVal90_5[i] \
                   )
    top10Error = min(errorVal90_1[i] \
                    ,errorVal90_2[i] \
                    ,errorVal90_3[i] \
                    ,errorVal90_4[i] \
                    ,errorVal90_5[i] \
                    ,errorVal90_6[i] \
                    ,errorVal90_7[i] \
                    ,errorVal90_8[i] \
                    ,errorVal90_9[i] \
                    ,errorVal90_10[i] \
                   )
    min90_1.append(top1Error)
    min90_3.append(top3Error)
    min90_5.append(top5Error)
    min90_10.append(top10Error)

In [ ]:
print(mean(min90_1))
print(mean(min90_3))
print(mean(min90_5))
print(mean(min90_10))

In [ ]:
plt.hist(min90_3)

In [ ]:
plt.hist(min90_5)

In [ ]:
min120_1 =[]
min120_3 =[]
min120_5 =[]
min120_10 =[]

for i in range(len(errorVal120_1)):
    top1Error = errorVal120_1[i]
    
    top3Error = min(errorVal120_1[i] \
                    ,errorVal120_2[i] \
                    ,errorVal120_3[i] \
                   )
    top5Error = min(errorVal120_1[i] \
                    ,errorVal120_2[i] \
                    ,errorVal120_3[i] \
                    ,errorVal120_4[i] \
                    ,errorVal120_5[i] \
                   )
    top10Error = min(errorVal120_1[i] \
                    ,errorVal120_2[i] \
                    ,errorVal120_3[i] \
                    ,errorVal120_4[i] \
                    ,errorVal120_5[i] \
                    ,errorVal120_6[i] \
                    ,errorVal120_7[i] \
                    ,errorVal120_8[i] \
                    ,errorVal120_9[i] \
                    ,errorVal120_10[i] \
                   )
    min120_1.append(top1Error)
    min120_3.append(top3Error)
    min120_5.append(top5Error)
    min120_10.append(top10Error)

In [ ]:
print(mean(min120_1))
print(mean(min120_3))
print(mean(min120_5))
print(mean(min120_10))

In [ ]:
plt.title("Histogram of Top 3 Error After 120 Minutes")
plt.xlabel("Error in KM")
plt.hist(min120_3)
figName = errorHistDest + "Top3Err120Min.png"
plt.savefig(figName)

In [ ]:
plt.title("Histogram of Top 5 Error After 120 Minutes")
plt.xlabel("Error in KM")
plt.hist(min120_5)
figName = errorHistDest + "Top5Err120Min.png"
plt.savefig(figName)

In [ ]:
min150_1 =[]
min150_3 =[]
min150_5 =[]
min150_10 =[]

for i in range(len(errorVal150_1)):
    top1Error = errorVal150_1[i]
    
    top3Error = min(errorVal150_1[i] \
                    ,errorVal150_2[i] \
                    ,errorVal150_3[i] \
                   )
    top5Error = min(errorVal150_1[i] \
                    ,errorVal150_2[i] \
                    ,errorVal150_3[i] \
                    ,errorVal150_4[i] \
                    ,errorVal150_5[i] \
                   )
    top10Error = min(errorVal150_1[i] \
                    ,errorVal150_2[i] \
                    ,errorVal150_3[i] \
                    ,errorVal150_4[i] \
                    ,errorVal150_5[i] \
                    ,errorVal150_6[i] \
                    ,errorVal150_7[i] \
                    ,errorVal150_8[i] \
                    ,errorVal150_9[i] \
                    ,errorVal150_10[i] \
                   )
    min150_1.append(top1Error)
    min150_3.append(top3Error)
    min150_5.append(top5Error)
    min150_10.append(top10Error)

In [ ]:
print(mean(min150_1))
print(mean(min150_3))
print(mean(min150_5))
print(mean(min150_10))

In [ ]:
plt.hist(min150_3)

In [ ]:
plt.hist(min150_5)

In [ ]:
min180_1 =[]
min180_3 =[]
min180_5 =[]
min180_10 =[]

for i in range(len(errorVal180_1)):
    top1Error = errorVal180_1[i]
    
    top3Error = min(errorVal180_1[i] \
                    ,errorVal180_2[i] \
                    ,errorVal180_3[i] \
                   )
    top5Error = min(errorVal180_1[i] \
                    ,errorVal180_2[i] \
                    ,errorVal180_3[i] \
                    ,errorVal180_4[i] \
                    ,errorVal180_5[i] \
                   )
    top10Error = min(errorVal180_1[i] \
                    ,errorVal180_2[i] \
                    ,errorVal180_3[i] \
                    ,errorVal180_4[i] \
                    ,errorVal180_5[i] \
                    ,errorVal180_6[i] \
                    ,errorVal180_7[i] \
                    ,errorVal180_8[i] \
                    ,errorVal180_9[i] \
                    ,errorVal180_10[i] \
                   )
    min180_1.append(top1Error)
    min180_3.append(top3Error)
    min180_5.append(top5Error)
    min180_10.append(top10Error)

In [ ]:
print(mean(min180_1))
print(mean(min180_3))
print(mean(min180_5))
print(mean(min180_10))

In [ ]:
plt.title("Histogram of Top 3 Error After 180 Minutes")
plt.xlabel("Error in KM")
plt.hist(min180_3)
figName = errorHistDest + "Top3Err180Min.png"
plt.savefig(figName)

In [ ]:
plt.title("Histogram of Top 5 Error After 180 Minutes")
plt.xlabel("Error in KM")
plt.hist(min180_5)
figName = errorHistDest + "Top5Err180Min.png"
plt.savefig(figName)

In [ ]:
min210_1 =[]
min210_3 =[]
min210_5 =[]
min210_10 =[]

for i in range(len(errorVal210_1)):
    top1Error = errorVal210_1[i]
    
    top3Error = min(errorVal210_1[i] \
                    ,errorVal210_2[i] \
                    ,errorVal210_3[i] \
                   )
    top5Error = min(errorVal210_1[i] \
                    ,errorVal210_2[i] \
                    ,errorVal210_3[i] \
                    ,errorVal210_4[i] \
                    ,errorVal210_5[i] \
                   )
    top10Error = min(errorVal210_1[i] \
                    ,errorVal210_2[i] \
                    ,errorVal210_3[i] \
                    ,errorVal210_4[i] \
                    ,errorVal210_5[i] \
                    ,errorVal210_6[i] \
                    ,errorVal210_7[i] \
                    ,errorVal210_8[i] \
                    ,errorVal210_9[i] \
                    ,errorVal210_10[i] \
                   )
    min210_1.append(top1Error)
    min210_3.append(top3Error)
    min210_5.append(top5Error)
    min210_10.append(top10Error)

In [ ]:
print(mean(min210_1))
print(mean(min210_3))
print(mean(min210_5))
print(mean(min210_10))

In [ ]:
plt.hist(min210_3)

In [ ]:
plt.hist(min210_5)

In [ ]:
min240_1 =[]
min240_3 =[]
min240_5 =[]
min240_10 =[]

for i in range(len(errorVal240_1)):
    top1Error = errorVal240_1[i]
    
    top3Error = min(errorVal240_1[i] \
                    ,errorVal240_2[i] \
                    ,errorVal240_3[i] \
                   )
    top5Error = min(errorVal240_1[i] \
                    ,errorVal240_2[i] \
                    ,errorVal240_3[i] \
                    ,errorVal240_4[i] \
                    ,errorVal240_5[i] \
                   )
    top10Error = min(errorVal240_1[i] \
                    ,errorVal240_2[i] \
                    ,errorVal240_3[i] \
                    ,errorVal240_4[i] \
                    ,errorVal240_5[i] \
                    ,errorVal240_6[i] \
                    ,errorVal240_7[i] \
                    ,errorVal240_8[i] \
                    ,errorVal240_9[i] \
                    ,errorVal240_10[i] \
                   )
    min240_1.append(top1Error)
    min240_3.append(top3Error)
    min240_5.append(top5Error)
    min240_10.append(top10Error)

In [ ]:
print(mean(min240_1))
print(mean(min240_3))
print(mean(min240_5))
print(mean(min240_10))

In [ ]:
plt.title("Histogram of Top 3 Error After 240 Minutes")
plt.xlabel("Error in KM")
plt.hist(min240_3)
figName = errorHistDest + "Top3Err240Min.png"
plt.savefig(figName)

In [ ]:
plt.title("Histogram of Top 5 Error After 240 Minutes")
plt.xlabel("Error in KM")
plt.hist(min240_5)
figName = errorHistDest + "Top5Err240Min.png"
plt.savefig(figName)

In [ ]:
top1List = [mean(min30_1) \
            ,mean(min60_1) \
            ,mean(min90_1) \
            ,mean(min120_1) \
            ,mean(min150_1) \
            ,mean(min180_1) \
            ,mean(min210_1) \
            ,mean(min240_1) \
           ]

top3List = [mean(min30_3) \
            ,mean(min60_3) \
            ,mean(min90_3) \
            ,mean(min120_3) \
            ,mean(min150_3) \
            ,mean(min180_3) \
            ,mean(min210_3) \
            ,mean(min240_3) \
           ]

top5List = [mean(min30_5) \
            ,mean(min60_5) \
            ,mean(min90_5) \
            ,mean(min120_5) \
            ,mean(min150_5) \
            ,mean(min180_5) \
            ,mean(min210_5) \
            ,mean(min240_5) \
           ]

top10List = [mean(min30_10) \
            ,mean(min60_10) \
            ,mean(min90_10) \
            ,mean(min120_10) \
            ,mean(min150_10) \
            ,mean(min180_10) \
            ,mean(min210_10) \
            ,mean(min240_10) \
           ]

In [ ]:
def format_func(value, tick_number):
    tempTick = (value*30) + 30
    ret = "%d"%(tempTick)
    return ret

In [ ]:
fig = plt.figure()
ax = fig.add_subplot()
ax.set_title("Average Error Value")
ax.set_ylabel("Distance in KM")
ax.set_xlabel("Time in Minutes")
ax.plot(top1List,label = "Top1")
ax.plot(top3List,label = "Top3")
ax.plot(top5List,label = "Top5")
ax.plot(top10List,label = "Top10")
# ax.set_ylim(0,40)
ax.xaxis.set_major_formatter(plt.FuncFormatter(format_func))
ax.legend()
fig.savefig(erroPlotName)